###### Cory Melendez
###### Natural Language Processing Project
###### https://github.com/cmelende/NLPProject.git
###### 12/11/20

In [1]:
import pandas as pd

In [3]:
data_source = './data/Tweets.csv'
pf = pd.read_csv(data_source)
pf.shape

(14640, 15)